In [3]:
import cv2
import mediapipe as mp
import copy
import itertools
from tensorflow import keras
import numpy as np
import pandas as pd
import string

# Load the saved model
model = keras.models.load_model("model.h5")

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

alphabet = ['1','2','3','4','5','6','7','8','9'] + list(string.ascii_uppercase)

def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]
    landmark_point = []

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)
    base_x, base_y = 0, 0

    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] -= base_x
        temp_landmark_list[index][1] -= base_y

    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

# Webcam input
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, frame = cap.read()
        frame = cv2.flip(frame, 1)
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Define square detection box
        box_size = 300
        frame_h, frame_w = frame.shape[:2]
        x1 = frame_w // 2 - box_size // 2
        y1 = frame_h // 2 - box_size // 2
        x2 = x1 + box_size
        y2 = y1 + box_size

        # Draw box on frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Crop the region of interest (ROI)
        roi = frame[y1:y2, x1:x2]
        image = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        debug_image = copy.deepcopy(roi)

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                pre_processed_landmark_list = pre_process_landmark(landmark_list)

                mp_drawing.draw_landmarks(
                    roi,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                df = pd.DataFrame(pre_processed_landmark_list).transpose()
                predictions = model.predict(df, verbose=0)
                predicted_classes = np.argmax(predictions, axis=1)
                label = alphabet[predicted_classes[0]]

                # Draw predicted label
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)
                print(label)
                print("------------------------")

        cv2.imshow('Indian sign language detector', frame)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


8
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
Z
------------------------
N
------------------------
N
------------------------
G
------------------------
G
------------------------
A
------------------------
A
------------------------
A
------------------------
A
------------------------
A
------------------------
A

In [30]:
%run lettersremember.py